In [5]:
%matplotlib inline
%config InlineBackend.figure_format='svg'
#%config InlineBackend.figure_format='retina'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.stats import LombScargle

from scipy import integrate
import astropy
from astropy.time import Time

In [4]:



%matplotlib inline
plt.style.use('seaborn-whitegrid')


import numpy as np
from matplotlib import pyplot as plt
from astroML.time_series import lomb_scargle, lomb_scargle_BIC, lomb_scargle_bootstrap

#----------------------------------------------------------------------
# This function adjusts matplotlib settings for a uniform feel in the textbook.
# Note that with usetex=True, fonts are rendered with LaTeX.  This may
# result in an error if LaTeX is not installed on your system.  In that case,
# you can set usetex to False.
from astroML.plotting import setup_text_plots

ModuleNotFoundError: No module named 'astroML'

In [7]:
data3c274=pd.read_excel('/Users/brettlv/blog/IHEP_sync/data/lombscargledata/3c274.xlsx')

In [8]:
data3c274

,BAND,day,month,year,TIME,OBSERV.,F(GHz),FLUX(JY),ERROR,FLUX P.I.
0,1mm,7,Apr,2003,05:27:00,SMA,235.60,1.940,0.107,mgurwell
1,1mm,12,Aug,2003,05:53:00,SMA,236.00,2.032,0.308,mgurwell
2,1mm,27,Jan,2004,08:57:00,SMA,224.75,1.388,0.094,mgurwell
3,1mm,9,Feb,2004,06:25:00,SMA,226.30,1.629,0.247,mgurwell
4,1mm,5,Mar,2004,07:05:00,SMA,226.30,1.689,0.130,mgurwell
...,...,...,...,...,...,...,...,...,...,...
205,1mm,10,May,2018,08:01:00,SMA,225.54,1.305,0.067,mgurwell
206,1mm,29,May,2018,05:31:00,SMA,237.00,1.189,0.072,mgurwell
207,1mm,5,Jun,2018,05:16:00,SMA,235.56,1.230,0.062,mgurwell
208,1mm,21,Aug,2018,04:01:00,SMA,223.52,1.347,0.089,mgurwell


In [9]:
from astropy.time import Time
from astropy.time import TimeYearDayTime
from datetime import datetime
from time import strftime,strptime
import calendar
from dateutil.parser import parse

In [10]:
day=data3c274['day']
month=data3c274['month']
year=data3c274['year']
time=data3c274['TIME']

In [11]:
month_number=[]
for i in month.values:
    month_number.append(strptime(i,'%b').tm_mon)

In [12]:
time.values[0]

datetime.time(5, 27)

In [13]:
h=[]
m=[]
for i in time.values:
    h.append(i.hour)
    m.append(i.minute)
    #h.append(i.split(":")[0])
    #m.append(i.split(":")[1])

In [14]:
data3c274['mon_num']=month_number
data3c274['hour']=h
data3c274['minute']=m

In [17]:
timesut=[]
for i,(d_,m_,y_,h_,mi_) in enumerate(zip(day,month_number,year,h,m)):                 
    timesut.append(Time(datetime(y_,m_,d_,h_,mi_,0),scale='utc').mjd)    
data3c274['mjd']=timesut

In [18]:
data3c274

,BAND,day,month,year,TIME,OBSERV.,F(GHz),FLUX(JY),ERROR,FLUX P.I.,mon_num,hour,minute,mjd
0,1mm,7,Apr,2003,05:27:00,SMA,235.60,1.940,0.107,mgurwell,4,5,27,52736.227083
1,1mm,12,Aug,2003,05:53:00,SMA,236.00,2.032,0.308,mgurwell,8,5,53,52863.245139
2,1mm,27,Jan,2004,08:57:00,SMA,224.75,1.388,0.094,mgurwell,1,8,57,53031.372917
3,1mm,9,Feb,2004,06:25:00,SMA,226.30,1.629,0.247,mgurwell,2,6,25,53044.267361
4,1mm,5,Mar,2004,07:05:00,SMA,226.30,1.689,0.130,mgurwell,3,7,5,53069.295139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,1mm,10,May,2018,08:01:00,SMA,225.54,1.305,0.067,mgurwell,5,8,1,58248.334028
206,1mm,29,May,2018,05:31:00,SMA,237.00,1.189,0.072,mgurwell,5,5,31,58267.229861
207,1mm,5,Jun,2018,05:16:00,SMA,235.56,1.230,0.062,mgurwell,6,5,16,58274.219444
208,1mm,21,Aug,2018,04:01:00,SMA,223.52,1.347,0.089,mgurwell,8,4,1,58351.167361


In [34]:
data3c279=pd.read_excel('/Users/brettlv/blog/IHEP_sync/file_to_mac/data/3c279.xlsx')
day=data3c279['day']
month=data3c279['month']
year=data3c279['year']
time=data3c279['TIME']
month_number=[]
for i in month.values:
    month_number.append(strptime(i,'%b').tm_mon)
h=[]
m=[]
for i in time.values:
    h.append(i.hour)
    m.append(i.minute)
    #h.append(i.split(":")[0])
    #m.append(i.split(":")[1])  
    
data3c279['mon_num']=month_number
data3c279['hour']=h
data3c279['minute']=m   
timesut=[]
for i,(d_,m_,y_,h_,mi_) in enumerate(zip(day,month_number,year,h,m)):                 
    timesut.append(Time(datetime(y_,m_,d_,h_,mi_,0),scale='ut1').mjd)    
data3c279['mjd']=timesut
#data3c279.to_csv('/Users/brettlv/blog/IHEP_sync/file_to_mac/data/3c279time.csv')  

In [36]:
data3c345=pd.read_excel('/Users/brettlv/blog/IHEP_sync/file_to_mac/data/3c345.xlsx')
day=data3c345['day']
month=data3c345['month']
year=data3c345['year']
time=data3c345['TIME']
month_number=[]
for i in month.values:
    month_number.append(strptime(i,'%b').tm_mon)
h=[]
m=[]
for i in time.values:
    h.append(i.hour)
    m.append(i.minute)
    #h.append(i.split(":")[0])
    #m.append(i.split(":")[1])  
    
data3c345['mon_num']=month_number
data3c345['hour']=h
data3c345['minute']=m   
timesut=[]
for i,(d_,m_,y_,h_,mi_) in enumerate(zip(day,month_number,year,h,m)):                 
    timesut.append(Time(datetime(y_,m_,d_,h_,mi_,0),scale='ut1').mjd)    
data3c345['mjd']=timesut
#data3c345.to_csv('/Users/brettlv/blog/IHEP_sync/file_to_mac/data/3c345time.csv') 

In [37]:
data3c454_3=pd.read_excel('/Users/brettlv/blog/IHEP_sync/file_to_mac/data/3c454.3.xlsx')
day=data3c454_3['day']
month=data3c454_3['month']
year=data3c454_3['year']
time=data3c454_3['TIME']
month_number=[]
for i in month.values:
    month_number.append(strptime(i,'%b').tm_mon)
h=[]
m=[]
for i in time.values:
    h.append(i.hour)
    m.append(i.minute)
    #h.append(i.split(":")[0])
    #m.append(i.split(":")[1])  
    
data3c454_3['mon_num']=month_number
data3c454_3['hour']=h
data3c454_3['minute']=m   
timesut=[]
for i,(d_,m_,y_,h_,mi_) in enumerate(zip(day,month_number,year,h,m)):                 
    timesut.append(Time(datetime(y_,m_,d_,h_,mi_,0),scale='ut1').mjd)    
data3c454_3['mjd']=timesut
#data3c454_3.to_csv('/Users/brettlv/blog/IHEP_sync/file_to_mac/data/3c454_3time.csv') 